In [ ]:
import re
import json

def generate_response(prompt):
    # missing pipeline, model, and tokenizer because this was run on sagemaker with 7a
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200) 
    
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    response = result[0]['generated_text']
    return response

def gen_prompt_response_pairs(profile):
    # Generate Prompt
    def clean_question(question):
        return re.sub(r'\s*\(.*?\)', '', question).strip()

    def approach_1():
        name = 'Naive approach, include all text'
        profile_str = ''
        for question, answer in profile.items():
            question = clean_question(question)
            profile_str += f'{question}: {answer}\n'
            
        prompt = (
            f"Consider the following question answer pairs:\n{profile_str}\n"
            "Based on those answers, which county in the United States would be the best to live in?\n\n"
        )
        return (name, prompt)
    
    def approach_2():
        name = 'Naive approach less questions'
        count = 0
        profile_str = ''
        for question, answer in profile.items():
            if count >= 2:
                break
            question = clean_question(question)
            profile_str += f'{question}: {answer}\n'
            count += 1
            
        prompt = (
            f"Consider the following question answer pairs:\n{profile_str}\n"
            "Based on those answers, which county in the United States would be the best to live in?\n\n"
        )
        return (name, prompt)
    
    def approach_5():
        name = 'Sentence Approach'
        qa_str = 'I prefer to live somewhere like this: '
        counter = 1
        for question, answer in profile.items():
            qa_str += f'{answer}. '
        prompt = (
            f"{qa_str}recommend me 3 counties that match this"
        )
        return (name, prompt)
    
    approachs = [approach_1(), approach_2(), approach_5()]
    
    prompt_response_pairs = {}
    for approach_name, prompt in approachs:
        response = generate_response(prompt)
        prompt_response_pairs[approach_name] = [prompt, response]
        
    return prompt_response_pairs

In [ ]:
user_profiles = []
urban_professional = ['Urban Professional',{
    "What type of terrain do you prefer to live in (e.g., forests, open water, developed areas)?": "Developed areas",
    "Are you looking for a county with a higher or lower population density (e.g., densely populated urban areas or more sparsely populated rural areas)?": "Higher population density",
    "What is your preferred demographic makeup of the community you live in (e.g., predominantly White, African American, Hispanic)?": "Predominantly White",
    "Is it important for you to live in an area with a lower or higher crime rate (e.g., low crime suburban areas or higher crime urban areas)?": "Lower crime rate",
    "What is your budget range for housing, specifically in terms of fair market rent for different bedroom sizes (e.g., $700 for one bedroom, $830 for two bedrooms, $1047 for three bedrooms, $1425 for four bedrooms)?": "$830 for two bedrooms",
    "Do you need access to agricultural land (e.g., areas with significant pasture/hay or cultivated crops)?": "No",
    "How important is proximity to natural features (e.g., areas with significant deciduous or evergreen forests, or regions with extensive woody or herbaceous wetlands)?": "Not important",
    "Are you looking for an area with more developed open space or one with more natural terrain (e.g., shrub/scrub, grassland/herbaceous)?": "More developed open space",
    "What median family income range are you comfortable with in the community you live in (e.g., communities with a median family income around $65,700)?": "Around $75,000"}]

rural_family = ['Rural Family',{
    "What type of terrain do you prefer to live in (e.g., forests, open water, developed areas)?": "Forests",
    "Are you looking for a county with a higher or lower population density (e.g., densely populated urban areas or more sparsely populated rural areas)?": "Lower population density",
    "What is your preferred demographic makeup of the community you live in (e.g., predominantly White, African American, Hispanic)?": "Predominantly African American",
    "Is it important for you to live in an area with a lower or higher crime rate (e.g., low crime suburban areas or higher crime urban areas)?": "Lower crime rate",
    "What is your budget range for housing, specifically in terms of fair market rent for different bedroom sizes (e.g., $700 for one bedroom, $830 for two bedrooms, $1047 for three bedrooms, $1425 for four bedrooms)?": "$1047 for three bedrooms",
    "Do you need access to agricultural land (e.g., areas with significant pasture/hay or cultivated crops)?": "Yes",
    "How important is proximity to natural features (e.g., areas with significant deciduous or evergreen forests, or regions with extensive woody or herbaceous wetlands)?": "Very important",
    "Are you looking for an area with more developed open space or one with more natural terrain (e.g., shrub/scrub, grassland/herbaceous)?": "More natural terrain",
    "What median family income range are you comfortable with in the community you live in (e.g., communities with a median family income around $65,700)?": "Around $65,700"}]

young_professional = ['Young Professional',{
    "What type of terrain do you prefer to live in (e.g., forests, open water, developed areas)?": "Developed areas",
    "Are you looking for a county with a higher or lower population density (e.g., densely populated urban areas or more sparsely populated rural areas)?": "Higher population density",
    "What is your preferred demographic makeup of the community you live in (e.g., predominantly White, African American, Hispanic)?": "Predominantly Hispanic",
    "Is it important for you to live in an area with a lower or higher crime rate (e.g., low crime suburban areas or higher crime urban areas)?": "Higher crime rate",
    "What is your budget range for housing, specifically in terms of fair market rent for different bedroom sizes (e.g., $700 for one bedroom, $830 for two bedrooms, $1047 for three bedrooms, $1425 for four bedrooms)?": "$700 for one bedroom",
    "Do you need access to agricultural land (e.g., areas with significant pasture/hay or cultivated crops)?": "No",
    "How important is proximity to natural features (e.g., areas with significant deciduous or evergreen forests, or regions with extensive woody or herbaceous wetlands)?": "Not important",
    "Are you looking for an area with more developed open space or one with more natural terrain (e.g., shrub/scrub, grassland/herbaceous)?": "More developed open space",
    "What median family income range are you comfortable with in the community you live in (e.g., communities with a median family income around $65,700)?": "Around $50,000"}]

retiree = ['Retiree',{
    "What type of terrain do you prefer to live in (e.g., forests, open water, developed areas)?": "Open water",
    "Are you looking for a county with a higher or lower population density (e.g., densely populated urban areas or more sparsely populated rural areas)?": "Lower population density",
    "What is your preferred demographic makeup of the community you live in (e.g., predominantly White, African American, Hispanic)?": "Predominantly White",
    "Is it important for you to live in an area with a lower or higher crime rate (e.g., low crime suburban areas or higher crime urban areas)?": "Lower crime rate",
    "What is your budget range for housing, specifically in terms of fair market rent for different bedroom sizes (e.g., $700 for one bedroom, $830 for two bedrooms, $1047 for three bedrooms, $1425 for four bedrooms)?": "$1425 for four bedrooms",
    "Do you need access to agricultural land (e.g., areas with significant pasture/hay or cultivated crops)?": "No",
    "How important is proximity to natural features (e.g., areas with significant deciduous or evergreen forests, or regions with extensive woody or herbaceous wetlands)?": "Very important",
    "Are you looking for an area with more developed open space or one with more natural terrain (e.g., shrub/scrub, grassland/herbaceous)?": "More natural terrain",
    "What median family income range are you comfortable with in the community you live in (e.g., communities with a median family income around $65,700)?": "Around $60,000"}]

middle_class_fam = ['Middle Class Family', {"What type of terrain do you prefer to live in (e.g., forests, open water, developed areas)?": "Forests",
    "Are you looking for a county with a higher or lower population density (e.g., densely populated urban areas or more sparsely populated rural areas)?": "Moderate population density",
    "What is your preferred demographic makeup of the community you live in (e.g., predominantly White, African American, Hispanic)?": "Mixed",
    "Is it important for you to live in an area with a lower or higher crime rate (e.g., low crime suburban areas or higher crime urban areas)?": "Lower crime rate",
    "What is your budget range for housing, specifically in terms of fair market rent for different bedroom sizes (e.g., $700 for one bedroom, $830 for two bedrooms, $1047 for three bedrooms, $1425 for four bedrooms)?": "$1047 for three bedrooms",
    "Do you need access to agricultural land (e.g., areas with significant pasture/hay or cultivated crops)?": "No",
    "How important is proximity to natural features (e.g., areas with significant deciduous or evergreen forests, or regions with extensive woody or herbaceous wetlands)?": "Important",
    "Are you looking for an area with more developed open space or one with more natural terrain (e.g., shrub/scrub, grassland/herbaceous)?": "More natural terrain",
    "What median family income range are you comfortable with in the community you live in (e.g., communities with a median family income around $65,700)?": "Around $70,000"}]

user_profiles.append(urban_professional)
user_profiles.append(rural_family)
user_profiles.append(young_professional)
user_profiles.append(retiree)
user_profiles.append(middle_class_fam)

In [ ]:
profile_dict = {}

for profile in user_profiles:
    profile_name = profile[0]
    cur_pairs = gen_prompt_response_pairs(profile[1])
    
    approaches = {}
    for approach_name in cur_pairs:
        approaches[approach_name] = {'prompt' : cur_pairs[approach_name][0],
                                     'response' : cur_pairs[approach_name][1]}
    profile_dict[profile_name] = approaches
    
print(profile_dict)
    
with open('prompts_response_all.json', 'w') as f:
    json.dump(profile_dict, f, indent=4)